# Simulation processing: step 1

Goal: take the simulated hadrons and create an output file for fj clustering

In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import vector

In [2]:
simulation_dir = '/global/cfs/cdirs/m3246/ewitkowski/delphes_data'

#sim_code = "wp_wzpythia_forcms_charge-mz50.0-mw40.0-mwp1000_full"
sim_code = "wp_wzpythia_forcms_charge-mz70.0-mw60.0-mwp1200_full"

In [3]:
selected_tower_ET = h5py.File(f'{simulation_dir}/{sim_code}_Tower_ET.h5', 'r')['values']
selected_tower_eta = h5py.File(f'{simulation_dir}/{sim_code}_Tower_Eta.h5', 'r')['values']
selected_tower_phi = h5py.File(f'{simulation_dir}/{sim_code}_Tower_Phi.h5', 'r')['values']
selected_tower_E = h5py.File(f'{simulation_dir}/{sim_code}_Tower_E.h5', 'r')['values']

selected_muon_pT = h5py.File(f'{simulation_dir}/{sim_code}_Muon_PT.h5', 'r')['values']
selected_muon_eta = h5py.File(f'{simulation_dir}/{sim_code}_Muon_Eta.h5', 'r')['values']
selected_muon_phi = h5py.File(f'{simulation_dir}/{sim_code}_Muon_Phi.h5', 'r')['values']
selected_muon_charge = h5py.File(f'{simulation_dir}/{sim_code}_Muon_Charge.h5', 'r')['values']

In [4]:
print(selected_tower_ET.shape)

(8000, 4096)


In [5]:
muon_mass = 0.1056583755 # GeV
particles_to_fastjet = [22, 130, 211, 1, 2]


start_read, stop_read, chunk_size = 0, 8000, 8000
update_freq = int(chunk_size/10.0)

delta_R_isos = [0.3, 0.5]



In [6]:
current_chunk_start = start_read

while current_chunk_start < stop_read:
    
    
    current_chunk_stop = current_chunk_start + chunk_size
    print(f"Processing chunk from {current_chunk_start} to {current_chunk_stop}...")

    outfile_dimuons = f"/global/cfs/cdirs/m3246/rmastand/dimuonAD/data_post_fj/muons_only_{current_chunk_start}_{current_chunk_stop}_{sim_code}.dat"
    outfile_dimuons_isolation = f"/global/cfs/cdirs/m3246/rmastand/dimuonAD/data_post_fj/muons_iso_{current_chunk_start}_{current_chunk_stop}_{sim_code}.dat"
    outfile_hadrons = f"/global/cfs/cdirs/m3246/rmastand/dimuonAD/data_pre_fj/hadrons_only_{current_chunk_start}_{current_chunk_stop}_{sim_code}.dat"

    ofile_muons = open(outfile_dimuons, "w")
    ofile_iso = open(outfile_dimuons_isolation, "w")
    ofile_hadrons = open(outfile_hadrons, "w")
    num_rejects = 0
   
    for event in range(current_chunk_start, current_chunk_stop):

        if event % update_freq == 0:
            print(f"On event {event}...")

        # get the nonmuons
        loc_E = selected_tower_E[event]
        loc_eta = selected_tower_eta[event][loc_E != 0]
        loc_phi = selected_tower_phi[event][loc_E != 0]
        loc_ET = selected_tower_ET[event][loc_E != 0]
        loc_E = loc_E[loc_E != 0]    

        # get the muons
        loc_mu_pt_0 = selected_muon_pT[event]
        loc_mu_eta = selected_muon_eta[event][loc_mu_pt_0 != 0]
        loc_mu_phi = selected_muon_phi[event][loc_mu_pt_0 != 0]
        loc_mu_charge = selected_muon_charge[event][loc_mu_pt_0 != 0]
        loc_mu_pt_0 = loc_mu_pt_0[loc_mu_pt_0 != 0]
        
        # check that there is at least one muon and one antimuon
        num_muons = np.sum(loc_mu_charge==-1.)
        num_amuons = np.sum(loc_mu_charge==1)
        
        
        if (num_muons < 1) or (num_amuons < 1):
            num_rejects += 1
        
        else:
            
            # find the hardest muon / antimuon
            loc_mu_pt = loc_mu_pt_0[loc_mu_charge==-1]
            max_muon_pt = np.max(loc_mu_pt)
            hardest_muon_id = np.where(loc_mu_pt_0==max_muon_pt)[0][0]
            
            loc_amu_pt = loc_mu_pt_0[loc_mu_charge==1]
            max_amuon_pt = np.max(loc_amu_pt)
            hardest_amuon_id = np.where(loc_mu_pt_0==max_amuon_pt)[0][0]
            
            
            ofile_muons.write("#BEGIN\n")
            ofile_iso.write("#BEGIN\n")
            ofile_hadrons.write("#BEGIN\n")

            # get the muons
            # construct the muon 4-vector
            mu = vector.obj(pt = loc_mu_pt_0[hardest_muon_id], eta = loc_mu_eta[hardest_muon_id], phi = loc_mu_phi[hardest_muon_id], M = muon_mass)
            amu = vector.obj(pt = loc_mu_pt_0[hardest_amuon_id], eta = loc_mu_eta[hardest_amuon_id], phi = loc_mu_phi[hardest_amuon_id], M = muon_mass)
            dimu_system = mu + amu

            ofile_muons.write(f"{dimu_system.pt} {dimu_system.eta} {dimu_system.phi} {dimu_system.M}\n")

            # get the hadrons and calculate muon isolation
            isolations_mu, isolations_amu = {R:0 for R in delta_R_isos}, {R:0 for R in delta_R_isos}

            for particle_i in range(len(loc_E)):
                #if np.abs(loc_tower["pdgId_"][particle_i]) in particles_to_fastjet:
                                        
                    # write out the particle
                    particle_vector = vector.obj(pt = loc_ET[particle_i], eta = loc_eta[particle_i], phi = loc_phi[particle_i], M = 0)
                    ofile_hadrons.write(f"{particle_vector.px} {particle_vector.py} {particle_vector.pz} {particle_vector.E}\n")
                    
                    # calculate the isolation contribution to the hardest (a)muon
                    delta_R_mu = mu.deltaR(particle_vector)
                    delta_R_amu = amu.deltaR(particle_vector)
                    
                    for R in delta_R_isos:
                        if delta_R_mu <= R:
                            isolations_mu[R] += (particle_vector.pt)/(mu.pt)
                        if delta_R_amu <= R:
                            isolations_amu[R] += (particle_vector.pt)/(amu.pt)
                            
            iso_muons_line = ""
            for R in delta_R_isos:
                iso_muons_line += str(isolations_mu[R]) + " " 
            for R in delta_R_isos:
                iso_muons_line += str(isolations_amu[R]) + " " 
                   
            ofile_iso.write(f"{iso_muons_line}\n")

            ofile_muons.write("#END\n")
            ofile_iso.write("#END\n")
            ofile_hadrons.write("#END\n")

    ofile_muons.close()  
    ofile_iso.close() 
    ofile_hadrons.close()
    
    print(f"Done processing chunk.")
    print(f"{num_rejects} events without 2 muons")
    print("\n")
    
    current_chunk_start += chunk_size

print("Done completely!")



                                         

Processing chunk from 0 to 8000...
On event 0...
On event 800...
On event 1600...
On event 2400...
On event 3200...
On event 4000...
On event 4800...
On event 5600...
On event 6400...
On event 7200...
Done processing chunk.
2347 events without 2 muons


Done completely!
